## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
import random
import statistics

# генерируем массив A с N=1млн случайных целых чисел на отрезке от 0 до 1000
N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]

# создаем массив B
B = [num + 100 for num in A]

# вычисляем среднее значение массива B
mean_B = statistics.mean(B)

print(mean_B)

600.01595


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [7]:
import pandas as pd
import numpy as np
import string

# создаем таблицу с 2млн строк и 4 столбцами со случайными числами
N = 2000000
df = pd.DataFrame(np.random.randint(0, 1000, size=(N, 4)), columns=list('ABCD'))

# добавляем столбец key с случайными выборками из первых пяти букв английского алфавита
df['key'] = np.random.choice(list(string.ascii_uppercase)[:5], N)

print(df.head())

     A    B    C    D key
0  488  823  308  687   E
1  564  546  560  116   E
2  340  898  110  174   C
3  901  798  183  599   A
4  369   42  249  248   D


## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [31]:
import pandas as pd
import time


def calculate_mean_rating_a(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews.iterrows():
        if pd.to_datetime(row['date']).year == 2010:
            ratings_sum += row['rating']
            count += 1
    return ratings_sum / count


def calculate_mean_rating_b(reviews):
    count = 0
    ratings_sum = 0
    for _, row in reviews[reviews['date'].dt.year == 2010].iterrows():
        ratings_sum += row['rating']
        count += 1
    return ratings_sum / count


def calculate_mean_rating_c(reviews):
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()


recipes = pd.read_csv('recipes_sample.csv', index_col=0)
reviews = pd.read_csv('reviews_sample.csv', index_col=0)

reviews['rating'] = reviews['rating'].astype(float)
reviews['date'] = pd.to_datetime(reviews['date'])


start_time = time.time()
mean_rating_a = calculate_mean_rating_a(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод A): {mean_rating_a} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_b = calculate_mean_rating_b(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод B): {mean_rating_b} Время выполнения: {end_time-start_time}")

start_time = time.time()
mean_rating_c = calculate_mean_rating_c(reviews)
end_time = time.time()
print(f"Средний рейтинг (метод C): {mean_rating_c} Время выполнения: {end_time-start_time}")

Средний рейтинг (метод A): 4.4544402182900615 Время выполнения: 3.1983492374420166
Средний рейтинг (метод B): 4.4544402182900615 Время выполнения: 0.3650822639465332
Средний рейтинг (метод C): 4.4544402182900615 Время выполнения: 0.006365060806274414


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [41]:
%load_ext line_profiler

%lprun -f calculate_mean_rating_a calculate_mean_rating_a(reviews)

Total time: 12.594 s

Function: calculate_mean_rating_a at line 5

Line       Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     5                                           def calculate_mean_rating_a(reviews):
     6         1          6.0      6.0      0.0      count = 0
     7         1          3.0      3.0      0.0      ratings_sum = 0
     8    126696  106369440.0    839.6     84.5      for _, row in reviews.iterrows():
     9    114602   18280001.0    159.5     14.5          if pd.to_datetime(row['date']).year == 2010:
    10     12094    1248927.0    103.3      1.0              ratings_sum += row['rating']
    11     12094      41319.0      3.4      0.0              count += 1
    12         1          4.0      4.0      0.0      return ratings_sum / count

In [ ]:
%lprun -f calculate_mean_rating_b calculate_mean_rating_b(reviews)

Total time: 1.18165 s

Function: calculate_mean_rating_b at line 15

Line       Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    15                                           def calculate_mean_rating_b(reviews):
    16         1          7.0      7.0      0.0      count = 0
    17         1          3.0      3.0      0.0      ratings_sum = 0
    18     12094   10246205.0    847.2     86.7      for _, row in reviews[reviews['date'].dt.year == 2010].iterrows():
    19     12094    1533351.0    126.8     13.0          ratings_sum += row['rating']
    20     12094      36950.0      3.1      0.3          count += 1
    21         1         12.0     12.0      0.0      return ratings_sum / count


In [ ]:
%lprun -f calculate_mean_rating_c calculate_mean_rating_c(reviews)

Total time: 0.0065571 s

Function: calculate_mean_rating_c at line 24

Line       Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    24                                           def calculate_mean_rating_c(reviews):
    25         1      65571.0  65571.0    100.0      return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [43]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [46]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

**Total time: 39.6466 s**

Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1          4.0      4.0      0.0      word_reviews = {}
     3    126679  113214133.0    893.7     28.6      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   28688792.0    226.5      7.2          recipe_id, review = row['recipe_id'], row['review']
     5    126679    3108442.0     24.5      0.8          words = review.split(' ')
     6   6792010   14647244.0      2.2      3.7          for word in words:
     7   6617066   22070375.0      3.3      5.6              if word not in word_reviews:
     8    174944     666738.0      3.8      0.2                  word_reviews[word] = []
     9   6792010   29539194.0      4.3      7.5              word_reviews[word].append(recipe_id)
    10                                               
    11         1          3.0      3.0      0.0      word_reviews_count = {}
    12    126679  112783072.0    890.3     28.4      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679   16290317.0    128.6      4.1          review = row['review']
    14    126679    3077365.0     24.3      0.8          words = review.split(' ')
    15   6792010   14080342.0      2.1      3.6          for word in words:
    16   6792010   38300167.0      5.6      9.7              word_reviews_count[word] = len(word_reviews[word])
    17         1          3.0      3.0      0.0      return word_reviews_count

Чтобы оптимизировать функцию, мы можем использовать один цикл для перебора строк и одновременного обновления словаря слов и связанных с ними идентификаторов просмотра. Затем мы можем один раз пройтись по словарю и подсчитать количество отзывов для каждого слова. Такой подход устраняет необходимость в нескольких циклах и вызовах функции len(). Вот оптимизированная реализация:

In [48]:
def get_word_reviews_count_optimized(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = set(review.split(' '))
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = set()
            word_reviews[word].add(recipe_id)
    
    word_reviews_count = {}
    for word in word_reviews:
        word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [49]:
%lprun -f get_word_reviews_count_optimized get_word_reviews_count_optimized(reviews)

**Total time: 22.4799 s**      

Function: get_word_reviews_count_optimized at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count_optimized(df):
     2         1          5.0      5.0      0.0      word_reviews = {}
     3    126679  119682724.0    944.8     53.2      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   29348042.0    231.7     13.1          recipe_id, review = row['recipe_id'], row['review']
     5    126679    7198896.0     56.8      3.2          words = set(review.split(' '))
     6   5387307   12834974.0      2.4      5.7          for word in words:
     7   5212363   19513779.0      3.7      8.7              if word not in word_reviews:
     8    174944     879073.0      5.0      0.4                  word_reviews[word] = set()
     9   5387307   33664959.0      6.2     15.0              word_reviews[word].add(recipe_id)
    10                                               
    11         1          3.0      3.0      0.0      word_reviews_count = {}
    12    174944     443649.0      2.5      0.2      for word in word_reviews:
    13    174944    1232615.0      7.0      0.5          word_reviews_count[word] = len(word_reviews[word])
    14         1          3.0      3.0      0.0      return word_reviews_count

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [71]:
import pandas as pd
import numpy as np
from numba import jit

# удаляем отзывы с нулевым рейтингом.
reviews = reviews[reviews['rating'] > 0]

def mape(ratings):
    """
    Calculate the MAPE for the given ratings.
    """
    mean_rating = np.mean(ratings)
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

# Версия 1: Без использования векторизованных операций и методов массивов numpy и без использования numba
def mape_v1(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mape_ = mape(ratings)
        mape_values.append(mape_)
    return mape_values

# Версия 2: Без использования векторизованных операций и методов массивов numpy, но с использованием numba
@jit(nopython=True)
def mape_numba(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using numba to speed up the
    calculations.
    """
    n = ratings.shape[0]
    abs_pct_error = np.empty(n)
    for i in range(n):
        abs_pct_error[i] = abs(ratings[i] - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v2(reviews):
    mape_values = []
    for recipe_id in reviews['recipe_id'].unique():
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = np.mean(ratings)
        mape_ = mape_numba(ratings.values, mean_rating)
        mape_values.append(mape_)
    return mape_values

                                   
# Версия 3: С использованием векторизованных операций и методов массивов numpy, но без использования numba
def mape_v3(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    mape_values = []
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating']
        mean_rating = mean_ratings[i]
        abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
        mape = np.mean(abs_pct_error) * 100
        mape_values.append(mape)
    return mape_values

# Версия 4: C использованием векторизованных операций и методов массивов numpy и numba
@jit(nopython=True)
def mape_numba_v2(ratings, mean_rating):
    """
    Calculate the MAPE for the given ratings using vectorization and numba to
    speed up the calculations.
    """
    abs_pct_error = np.abs(ratings - mean_rating) / mean_rating
    mape = np.mean(abs_pct_error) * 100
    return mape

def mape_v4(reviews):
    recipe_ids = reviews['recipe_id'].unique()
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    mape_values = np.empty(len(recipe_ids))
    for i, recipe_id in enumerate(recipe_ids):
        recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
        ratings = recipe_reviews['rating'].values
        mean_rating = mean_ratings[i]
        mape = mape_numba_v2(ratings, mean_rating)
        mape_values[i] = mape
    return mape_values



In [59]:
%lprun -f mape_v1 mape_v1(reviews)

**Total time: 36.2507 s**

Function: mape_v1 at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    25                                           def mape_v1(reviews):
    26         1          7.0      7.0      0.0      mape_values = []
    27     27440     200411.0      7.3      0.1      for recipe_id in reviews['recipe_id'].unique():
    28     27440  139745092.0   5092.8     38.5          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    29     27440   16363852.0    596.4      4.5          ratings = recipe_reviews['rating']
    30     27440  206002419.0   7507.4     56.8          mape_ = mape(ratings)
    31     27440     194814.0      7.1      0.1          mape_values.append(mape_)
    32         1          2.0      2.0      0.0      return mape_values

In [62]:
%lprun -f mape_v2 mape_v2(reviews)

**Total time: 18.5514 s**

Function: mape_v2 at line 48

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    48                                           def mape_v2(reviews):
    49         1          6.0      6.0      0.0      mape_values = []
    50     27440     178268.0      6.5      0.1      for recipe_id in reviews['recipe_id'].unique():
    51     27440  127306976.0   4639.5     68.6          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    52     27440   15225903.0    554.9      8.2          ratings = recipe_reviews['rating']
    53     27440   38582228.0   1406.1     20.8          mean_rating = np.mean(ratings)
    54     27440    4060366.0    148.0      2.2          mape_ = mape_numba(ratings.values, mean_rating)
    55     27440     160569.0      5.9      0.1          mape_values.append(mape_)
    56         1          2.0      2.0      0.0      return mape_values

In [67]:
%lprun -f mape_v4 mape_v4(reviews)

Total time: 15.5426 s

Function: mape_v4 at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    84                                           def mape_v4(reviews):
    85         1      27888.0  27888.0      0.0      recipe_ids = reviews['recipe_id'].unique()
    86         1     115180.0 115180.0      0.1      mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean').values
    87         1         39.0     39.0      0.0      mape_values = np.empty(len(recipe_ids))
    88     27440     222061.0      8.1      0.1      for i, recipe_id in enumerate(recipe_ids):
    89     27440  132584017.0   4831.8     85.3          recipe_reviews = reviews[reviews['recipe_id'] == recipe_id]
    90     27440   16843852.0    613.8     10.8          ratings = recipe_reviews['rating'].values
    91     27440     152456.0      5.6      0.1          mean_rating = mean_ratings[i]
    92     27440    5296185.0    193.0      3.4          mape = mape_numba_v2(ratings, mean_rating)
    93     27440     184743.0      6.7      0.1          mape_values[i] = mape
    94         1          2.0      2.0      0.0      return mape_values